In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import joblib
from pathlib import Path

from src import settings
from src.visualization.plot import line
#from src.data.make_knn_dataset import get_organ_labels
from src.visualization.templates import cmap_qualitative, cmap_quantitative
from src.evaluation.si_classifier import get_label_mapping
from src.utils.gather_pa_spectra_from_dataset import calculate_mean_spectrum
from src.utils.susi import ExperimentResults

# plot organ spectra

In [ ]:
spectra_file = settings.figures_dir / 'semantic_reflectance.csv'
df = pd.read_csv(spectra_file)
df = df[df.organ != 'gallbladder']

In [ ]:
def prepare_data(df: pd.DataFrame):
    to_replace = {
        'simulated_sampled': 'simulated',
        'small_bowel': 'small bowel',
        'inn': 'cINN',
        'unit': 'UNIT'
    }
    to_rename = {
        'wavelength': 'wavelength [nm]',
        'reflectance': 'reflectance [a.u.]',
        'dataset': 'data'
    }
    new_df = df.replace(to_replace, inplace=False)
    new_df.rename(to_rename, inplace=True, axis=1)
    to_rename.update(to_replace)
    return new_df, to_rename

In [ ]:
organs = get_organ_labels()['organ_labels']
organs = [o for o in organs if o != 'gallbladder']
organs = [' '.join(o.split('_')) for o in organs]
df_prepared, mapper = prepare_data(df)
fig, plot_data = line(
    data_frame=df_prepared,
    x=mapper.get("wavelength"),
    y=mapper.get('reflectance'),
    facet_col="organ",
    color=mapper.get("dataset"),
    facet_col_wrap=5,
    template="plotly_white",
    width=800,
    height=400,
    category_orders=dict(organ=organs, data=['real', 'simulated', 'cINN', 'UNIT']),
    facet_row_spacing=0.2,
    facet_col_spacing=0.05,
    color_discrete_map=cmap_qualitative,
    range_x=(900, 1000),
    range_y=(0.005, 0.015)
)

fig.update_layout(font=dict(size=12, family="Whitney Book"))
fig.update_xaxes(title_font=dict(size=12, family="Whitney Book"))
fig.update_yaxes(title_font=dict(size=12, family="Whitney Book"))
# fig.for_each_yaxis(lambda yaxis: yaxis.update(showticklabels=True))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.write_image(settings.figures_dir / 'manuscript' / 'organ_reflectance.pdf')

# plot confusion matrices

In [ ]:
stages = [
    'real',
    'sampled',
    'adapted_inn',
    'unit'
]
stage_mapper = dict(real='real', sampled='simulated', adapted_inn='cINN', unit='UNIT')
mapping = get_label_mapping()
for stage in stages:
    file = settings.results_dir / 'rf' / f"rf_classifier_matrix_{stage}.npz"
    data = np.load(file)
    matrix = data['matrix']
    labels = data['labels']
    names = [mapping.get(str(l)) for l in labels]
    fig = px.imshow(matrix,
                    text_auto='.2f',
                    color_continuous_scale=cmap_quantitative.get(stage_mapper.get(stage)),
                    zmin=0,
                    zmax=1,
                    template='plotly_white'
                    )
    fig.update_layout(font=dict(size=16))
    axis_ticks = dict(
            tickmode='array',
            tickvals=np.arange(0, len(names)),
            ticktext=names
        )
    fig.update_layout(
        xaxis=axis_ticks,
        yaxis=axis_ticks,
        coloraxis_colorbar=dict(
            title="probability",
            x=0.85,
            ticks="outside",
            ticksuffix="",
        )
    )
    fig.write_image(settings.figures_dir / 'manuscript' / f'rf_confusion_matrix_{stage}.pdf')

# plot photoacoustic images

In [ ]:
file = "/media/E130-Projekte/Photoacoustics/Projects/MICCAI_23/segmentation_task/real_data/validation/Study_70_Scan_13.npz"
data = np.load(file).get('reconstruction')

In [ ]:
fig = px.imshow(data[0, ...],
                template='plotly_white',
                color_continuous_scale="Electric")
fig.write_image(settings.figures_dir / 'manuscript' / 'reconstruction.pdf')

# plot photoacoustic spectra

In [ ]:
sources = [
    ('simulated', '/media/menjivar/Extreme SSD/MICCAI_23/results/photoacoustics/good_simulations/test'),
    ('real', '/media/menjivar/Extreme SSD/MICCAI_23/results/photoacoustics/real_images/validation'),
    ('inn', '/media/menjivar/Extreme SSD/MICCAI_23/results/photoacoustics/2023_01_23_22_47_44/testing/training'),
    ('unit', '/media/menjivar/Extreme SSD/MICCAI_23/results/photoacoustics/2023_02_18_15_05_06/testing/training'),
]

In [ ]:
data= {}
results = ExperimentResults()
for name, path in sources:
    files = Path(path).glob('*.npz')
    data = calculate_mean_spectrum(files)
    tissue_data = [('artery', data.get('artery_spectra_all')), ('vein', data.get('vein_spectra_all'))]
    for tissue, x in tissue_data:
        results.append(name="absorption", value=x.flatten())
        results.append(name="wavelength", value=np.repeat(settings.pai_wavelengths, x.shape[0]))
        results.append(name="data", value=[name for _ in x.flatten()])
        results.append(name="tissue", value=[tissue for _ in x.flatten()])

In [ ]:
df = results.get_df()
df, mapper = prepare_data(df)

In [ ]:
fig, _ = line(data_frame=df,
              x=mapper.get("wavelength"),
              y="absorption",
              facet_col="tissue",
              color="data",
              template="plotly_white",
              facet_col_spacing=0.05,
              color_discrete_map=cmap_qualitative,
           )
fig.update_layout(font=dict(size=12, family="Whitney Book"))
fig.update_xaxes(title_font=dict(size=12, family="Whitney Book"))
fig.update_yaxes(title_font=dict(size=12, family="Whitney Book"))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.write_image(settings.figures_dir / 'manuscript' / 'tissue_absorption.pdf')

# plot PAI PCA

In [ ]:
df = pd.read_csv(settings.results_dir / 'pca' / 'pai_pca.csv')
df = df[df['data'] != 'unit']
df, mapper = prepare_data(df)

In [ ]:
def trace_selector_2d_contour(tr):
    return hasattr(tr, 'ncontours')

In [ ]:
for tissue in df.tissue.unique():
    model_file = settings.results_dir / 'pca' / f'pai_pca_{tissue}.joblib'
    model = joblib.load(model_file)
    tmp = df[df['tissue'] == tissue].copy()
    tmp = tmp.rename({'pc_1': f"PC 1 [{round(model.explained_variance_ratio_[0]*100)}%]",
                       'pc_2': f"PC 2 [{round(model.explained_variance_ratio_[1]*100)}%]"},
                     axis=1)
    fig = px.density_contour(data_frame=tmp,
                             x=f"PC 1 [{round(model.explained_variance_ratio_[0]*100)}%]",
                             y=f"PC 2 [{round(model.explained_variance_ratio_[1]*100)}%]",
                             color="data",
                             histfunc="count",
                             histnorm="percent",
                             marginal_x='histogram',
                             marginal_y='histogram',
                             template="plotly_white",
                             color_discrete_map=cmap_qualitative,
                             range_x=(-0.3, 0.3),
                             range_y=(-0.1, 0.1),
                             )
    fig.update_traces(ncontours=5,
                      line=dict(width=2),
                      selector=trace_selector_2d_contour,
                      contours=dict(coloring="fill"),
                      opacity=0.4,
                      autocolorscale=False,
                      colorbar=None)
    fig.update_layout(font=dict(size=12, family="Whitney Book"))
    fig.update_xaxes(title_font=dict(size=12, family="Whitney Book"))
    fig.update_yaxes(title_font=dict(size=12, family="Whitney Book"))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.write_image(settings.figures_dir / 'manuscript' / f'pai_pca_{tissue}.pdf')